### Proyecto Individual Henry
**5_Sistema_Reco**  
Sistema de Recomendación
**Autor: Bioing. Urteaga Facundo Nahuel**  

**Resumen:** Este script comprende las siguientes etapas:

1. **Carga de librerías**
2. **Carga de datos (archivo .parquet)**
3. **Pre-procesamiento de dataframes para el análisis posterior**
4. **Primer entrenamiento del modelo (V1)**
5. **Segundo entrenamiento del modelo (V2)**
6. **Tercer entrenamiento del modelo (V3)**
7. **Cuarto entrenamiento del modelo (V4)**

In [260]:
### 1. Carga de librerías

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

Entrenamiento del primer modelo (V1)

In [271]:
### 2. Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

### 3. Pre-procesamiento de dataframes para el análisis posterior

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres'])
df_games_specs = df_games_specs.drop(columns=['specs'])
df_games_tags = df_games_tags.drop(columns=['tags'])

# Realiza un join de los DataFrames df1 y df2
merged_df_1 = pd.merge(df_games_names, df_games_genres, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_2 = pd.merge(merged_df_1, df_games_specs, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_final = pd.merge(merged_df_2, df_games_tags, on='item_id', how='inner')
#len(merged_df_final.columns)

### 4. Separo Info de nombre de variables dummies

#games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_id_names = merged_df_final[['item_id', 'app_name']]

### 5. Primer entrenamiento del modelo (V1)

n_neighbors = 6

nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

In [ ]:
# Indices de prueba. Juegos de diferentes características

# Counter Strike         |     10
# PES 2018               | 592580
# AGE III                | 105450
# Simcity 4              |  24780
# Tennis Elbow 2013      | 346470
# Civilization IV        |  16810
# Darksiders             |  50620
# Fallout NV             |  22380
# Dragon Age Origins     |  47810
# Star Wars Jedi Knight  |   6020
# NFS Shift              |  24870
# Final DOOM             |   2290
# Earthworm Jim          | 901147

In [272]:
### 6. TEST de modelo 

item_busqueda = 346470
index = games_dummies.index[games_id_names['item_id'] == item_busqueda][0]
#print(games_id_names["app_name"].iloc[index])

game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

#print(df_games_names.loc[ind[0][0:], "app_name"].values)

print("Juego Seleccionado")
print("-"*80)
print(games_id_names["app_name"].iloc[index])

if df_games_names.loc[ind[0][0], :].iloc[0] == item_busqueda:
    print("="*80)
    print("Juegos Recomendados")
    print("-"*80)
    print(df_games_names.loc[ind[0][1:6],  "app_name"])
else:
    print("="*80)
    print("Juegos Recomendados")
    print("-"*80)
    print(df_games_names.loc[ind[0][0:],  "app_name"])

Juego Seleccionado
--------------------------------------------------------------------------------
Tennis Elbow 2013
Juegos Recomendados
--------------------------------------------------------------------------------
27138                  Shin Samurai Jazz
25005    RONIN - Special Edition Upgrade
25004                     Genesis Online
9882                  Table Football Pro
12324         WWE 2K18 - Kurt Angle Pack
19575                          Burgers 2
Name: app_name, dtype: object


c:\Users\urtea\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Entrenamiento del segundo modelo (V2)

In [257]:
# Detecto categorías en specs, genres y labels que, a mi criterio, no aportan información al algoritmo

#df_games_specs.columns

# Encuentro que de acá podría solo dejar ['Mods','Online Multi-Player','Standing','Local Multi-Player','Room-Scale',
# 'Single-player', 'Windows Mixed Reality', 'Keyboard / Mouse','HTC Vive', 'Cross-Platform Multiplayer', 'Online Co-op', 'Seated',
# 'MMO','Co-op', 'Gamepad', 'Downloadable Content','Local Co-op','Multi-player']

#df_games_genres.columns

# Encuentro que de acá podría sacar Early Acces

#df_games_tags.columns[300:]

# De acá puedo sacar "Early Acces","Soundtrack"

# CAMBIOS PARA NUEVO ESTADO DE SISTEMA DE RECOMENDACION (V2)

#   * Eliminar las columnas nombradas
#   * Ponderar specs*1 genres*4 tags*0.5 (Por inspección, genres pondera mejor, luego specs y luegos tags, ya que tags es definido por usuarios y hay errores)

In [273]:
### 2. Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

# Vuelvo a realizar el SIST de RECO con estas modificaciones:

### 3. Selecciono solo las columnas de interés

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres','Early Access'])
df_games_specs = df_games_specs[['item_id','Mods','Online Multi-Player','Standing','Local Multi-Player','Room-Scale',
    'Single-player', 'Windows Mixed Reality', 'Keyboard / Mouse','HTC Vive', 'Cross-Platform Multiplayer', 'Online Co-op', 'Seated',
    'MMO','Co-op', 'Gamepad', 'Downloadable Content','Local Co-op','Multi-player']]
df_games_tags = df_games_tags.drop(columns=['tags',"Early Access","Soundtrack"])

### 4. Pondero categorías

df_games_specs[df_games_genres == 1] = 4
df_games_specs[df_games_specs == 1] = 1
df_games_tags[df_games_tags == 1] = 0.5

# Realiza un join de los DataFrames df1 y df2
merged_df_1 = pd.merge(df_games_names, df_games_genres, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_2 = pd.merge(merged_df_1, df_games_specs, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_final = pd.merge(merged_df_2, df_games_tags, on='item_id', how='inner')

### 5. Separo Info de nombre de variables dummies

#games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])
games_id_names = merged_df_final[['item_id', 'app_name']]

### 6. Segundo entrenamiento del modelo (V2)

n_neighbors = 6

nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)


C:\Users\urtea\AppData\Local\Temp\ipykernel_6800\564995624.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.5' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_games_tags[df_games_tags == 1] = 0.5


In [ ]:
# Indices de prueba. Juegos de diferentes características

# Counter Strike         |     10
# PES 2018               | 592580
# AGE III                | 105450
# Simcity 4              |  24780
# Tennis Elbow 2013      | 346470
# Civilization IV        |  16810
# Darksiders             |  50620
# Fallout NV             |  22380
# Dragon Age Origins     |  47810
# Star Wars Jedi Knight  |   6020
# NFS Shift              |  24870
# Final DOOM             |   2290
# Earthworm Jim          | 901147

In [274]:
### 7. TEST de modelo 

item_busqueda = 346470
index = games_dummies.index[games_id_names['item_id'] == item_busqueda][0]
#print(games_id_names["app_name"].iloc[index])

game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

#print(df_games_names.loc[ind[0][0:], "app_name"].values)

print("Juego Seleccionado")
print("-"*80)
print(games_id_names["app_name"].iloc[index])

if df_games_names.loc[ind[0][0], :].iloc[0] == item_busqueda:
    print("="*80)
    print("Juegos Recomendados")
    print("-"*80)
    print(df_games_names.loc[ind[0][1:6],  "app_name"])
else:
    print("="*80)
    print("Juegos Recomendados")
    print("-"*80)
    print(df_games_names.loc[ind[0][0:],  "app_name"])

Juego Seleccionado
--------------------------------------------------------------------------------
Tennis Elbow 2013
Juegos Recomendados
--------------------------------------------------------------------------------
27138                  Shin Samurai Jazz
25005    RONIN - Special Edition Upgrade
25004                     Genesis Online
14061                      Rytmik Studio
19575                          Burgers 2
9882                  Table Football Pro
Name: app_name, dtype: object


c:\Users\urtea\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [24]:
for iter in range(len(df_games_specs.columns)):
    if df_games_specs.iloc[30698, iter] != 0:
        nombre_columna = df_games_specs.columns[iter]
        print(nombre_columna)

# Por inspección encuentro que debo eliminar "Downloable Content" de df_games_specs para evitar que me recomiende DLCS y contenido descargable.
# También encuentro que puede recomendar juegos de cualquier año y eso puede ser algo no deseable

item_id
Single-player
Downloadable Content


In [ ]:
# 6. Tercer entrenamiento del modelo (V3)

# CAMBIOS PARA NUEVO ESTADO DE SISTEMA DE RECOMENDACION (V3)

#   * Eliminar Downloable Content
#   * Agregar y ponderar columna de decada del juego 

In [25]:
df_games_tec.columns

Index(['app_name', 'item_id', 'publisher', 'release_date', 'price',
       'developer', 'release_year'],
      dtype='object')

In [26]:
df_games_tec["release_year"].value_counts()

release_year
2017    9518
2016    6933
2015    4945
2014    2831
2013    1551
2012    1191
2011     589
2010     436
2009     337
2008     213
2007     167
2006     155
2005     102
2018      96
2003      84
2004      79
2001      69
1998      61
1999      55
2002      50
1997      42
2000      40
1996      39
1994      36
1993      34
1995      33
1991      16
1992      15
1990      12
1989      10
1987       9
1988       7
1984       4
1983       4
1985       3
1982       3
1981       3
2019       3
1970       2
1986       1
2021       1
1975       1
1980       1
Name: count, dtype: Int64

In [27]:
# Agrupo las fechas en décadas para que la influencia del año de cada juego sea mas flexible. Luego, genero variables dummies.

df_games_release_lustrum = df_games_tec[['item_id', 'release_year']].copy()

# Definir los límites de los lustros
bins = [0, 1999, 2005, 2010, 2015, 9999]
labels = ['before_2000', '2000_2005', '2005_2010', '2010_2015', 'after_2015']

# Dividir los años en lustros y crear variables dummies
df_games_release_lustrum['release_lustrum'] = pd.cut(df_games_release_lustrum['release_year'], bins=bins, labels=labels)
df_games_release_lustrum = pd.get_dummies(df_games_release_lustrum, columns=['release_lustrum'])
df_games_release_lustrum = df_games_release_lustrum.multiply(1)

# Eliminar la columna original de 'release_year'
df_games_release_lustrum.drop(columns=['release_year'], inplace=True)


In [28]:
df_games_release_lustrum["release_lustrum_2010_2015"].value_counts()

release_lustrum_2010_2015
0    21025
1    11107
Name: count, dtype: int64

In [41]:
df_games_tec.iloc[32131]

app_name        Maze Run VR
item_id            681550.0
publisher              None
release_date            NaT
price                  4.99
developer              None
release_year           <NA>
Name: 32131, dtype: object

In [40]:
df_games_release_lustrum["release_lustrum_after_2015"].value_counts()

release_lustrum_after_2015
1    16551
0    15581
Name: count, dtype: int64

In [33]:
# Mostrar el nuevo DataFrame
df_games_release_lustrum.tail()

,item_id,release_lustrum_before_2000,release_lustrum_2000_2005,release_lustrum_2005_2010,release_lustrum_2010_2015,release_lustrum_after_2015
32127,773640.0,0,0,0,0,1
32128,733530.0,0,0,0,0,1
32129,610660.0,0,0,0,0,1
32130,658870.0,0,0,0,0,1
32131,681550.0,0,0,0,0,0


In [42]:
# 2. Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

# Agrupo las fechas en décadas para que la influencia del año de cada juego sea mas flexible. Luego, genero variables dummies.

df_games_release_lustrum = df_games_tec[['item_id', 'release_year']].copy()

# Definir los límites de los lustros
bins = [0, 1999, 2005, 2010, 2015, 9999]
labels = ['before_2000', '2000_2005', '2005_2010', '2010_2015', 'after_2015']

# Dividir los años en lustros y crear variables dummies
df_games_release_lustrum['release_lustrum'] = pd.cut(df_games_release_lustrum['release_year'], bins=bins, labels=labels)
df_games_release_lustrum = pd.get_dummies(df_games_release_lustrum, columns=['release_lustrum'])
df_games_release_lustrum = df_games_release_lustrum.multiply(1)

# Eliminar la columna original de 'release_year'
df_games_release_lustrum.drop(columns=['release_year'], inplace=True)



# Vuelvo a realizar el SIST de RECO con estas modificaciones:
# OJO: Volver a cargar dfs

# Selecciono solo las columnas de interés

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres','Early Access'])
df_games_specs = df_games_specs[['item_id','Online Multi-Player','Local Multi-Player','Room-Scale',
    'Single-player', 'Keyboard / Mouse', 'Cross-Platform Multiplayer', 'Online Co-op', 'Seated',
    'MMO','Co-op', 'Gamepad','Local Co-op','Multi-player']]
df_games_tags = df_games_tags.drop(columns=['tags',"Early Access","Soundtrack"])

# Pondero categorías
df_games_release_lustrum[df_games_release_lustrum == 1] = 1 # Ponderación fuerte ya que es una sola columna con 1
df_games_specs[df_games_specs == 1] = 0.25
df_games_tags[df_games_tags == 1] = 1
df_games_genres[df_games_genres == 1] = 0.125

# Realiza un join de los DataFrames df1 y df2
merged_df_1 = pd.merge(df_games_names, df_games_genres, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_2 = pd.merge(merged_df_1, df_games_specs, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_3 = pd.merge(merged_df_2, df_games_release_lustrum, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_final = pd.merge(merged_df_3, df_games_tags, on='item_id', how='inner')

C:\Users\urtea\AppData\Local\Temp\ipykernel_6800\1979004.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.25' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_games_specs[df_games_specs == 1] = 0.25
C:\Users\urtea\AppData\Local\Temp\ipykernel_6800\1979004.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.125' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_games_genres[df_games_genres == 1] = 0.125


In [233]:
# Índices de juegos conocidos para luego poder evaluar el desempeño del modelo

# Juego Counter Strike 32103
# Juego de futbol 32108
# Juego de baseball 6001
# Juego de Formula 1 7013
# Juego Worms 7027

In [43]:
games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])

n_neighbors=6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

index = 32103
game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

print("Juego seleccionado")
print("="*80)
print(df_games_names.loc[ind[0][0], :])
print("Juegos recomendados")
print("="*80)
df_games_names.loc[ind[0][1:], :]

Juego seleccionado
item_id               10.0
app_name    Counter-Strike
Name: 32103, dtype: object
Juegos recomendados


c:\Users\urtea\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,item_id,app_name
16812,645130.0,Space Dream VR
17924,627030.0,Fantasy Grounds - Pathfinder RPG - Rise of the...
10390,672570.0,The Western Hunter
19863,540711.0,Assetto Corsa - Porsche Pack III
14682,728461.0,NASCAR Heat 2 - October Value Pack


In [ ]:
# 7. **Cuarto entrenamiento del modelo (V4)**

# CAMBIOS PARA NUEVO ESTADO DE SISTEMA DE RECOMENDACION (V4)

#   * Dividir cada variable dummie por la sumatoria total de etiquetas en cada categoría (genre, spec y tags)

In [44]:
df_games_tags.columns

Index(['item_id', 'Thriller', 'Philisophical', 'Superhero',
       'Massively Multiplayer', 'Diplomacy', 'Satire', 'Mature',
       'Side Scroller', 'Mystery Dungeon',
       ...
       'Word Game', 'Visual Novel', 'Experimental', 'Capitalism', 'Hex Grid',
       'Hacking', 'Video Production', 'Hunting', 'Turn-Based', 'Underwater'],
      dtype='object', length=338)

In [45]:
# 2. Carga de dataframes

df_games_tec = pd.read_parquet('df_games_tec.parquet')
df_games_genres = pd.read_parquet('df_games_genres.parquet')
df_games_specs = pd.read_parquet('df_games_specs.parquet')
df_games_tags = pd.read_parquet('df_games_tags.parquet')

# Selecciono solo las columnas de interés

df_games_names = df_games_tec[['item_id', 'app_name']]
df_games_genres = df_games_genres.drop(columns=['genres','Early Access'])
df_games_specs = df_games_specs[['item_id','Online Multi-Player','Local Multi-Player','Room-Scale',
    'Single-player', 'Keyboard / Mouse', 'Cross-Platform Multiplayer', 'Online Co-op', 'Seated',
    'MMO','Co-op', 'Gamepad','Local Co-op','Multi-player']]
df_games_tags = df_games_tags.drop(columns=['tags',"Early Access","Soundtrack"])

# Seleccionar solo las columnas de variables dummies
df_games_genres_dummies = df_games_genres.drop(columns=['item_id'])
df_games_specs_dummies = df_games_specs.drop(columns=['item_id'])
df_games_tags_dummies = df_games_tags.drop(columns=['item_id'])

# Sumar por fila la cantidad de variables dummies que son 1
suma_por_fila1 = df_games_genres_dummies.sum(axis=1)
suma_por_fila2 = df_games_specs_dummies.sum(axis=1)
suma_por_fila3 = df_games_tags_dummies.sum(axis=1)

# Dividir cada valor en la fila por la suma total (evitando la división por cero)
df_games_genres_dummies_dividido = df_games_genres_dummies.div(suma_por_fila1, axis=0)
df_games_specs_dummies_dividido = df_games_specs_dummies.div(suma_por_fila2, axis=0)
df_games_tags_dummies_dividido = df_games_tags_dummies.div(suma_por_fila3, axis=0)

# Reemplazar NaN con 0 si la suma por fila es 0
df_games_genres_dummies_dividido.fillna(0, inplace=True)
df_games_specs_dummies_dividido.fillna(0, inplace=True)
df_games_tags_dummies_dividido.fillna(0, inplace=True)

# Unir el DataFrame resultante con las columnas 'item_id' y 'genres'
df_games_genres_v4 = pd.concat([df_games_genres[['item_id']], df_games_genres_dummies_dividido], axis=1)
df_games_specs_v4 = pd.concat([df_games_specs[['item_id']], df_games_specs_dummies_dividido], axis=1)
df_games_tags_v4 = pd.concat([df_games_tags[['item_id']], df_games_tags_dummies_dividido], axis=1)


In [46]:
# Agrupo las fechas en décadas para que la influencia del año de cada juego sea mas flexible. Luego, genero variables dummies.

df_games_release_lustrum = df_games_tec[['item_id', 'release_year']].copy()

# Definir los límites de los lustros
bins = [0, 1999, 2005, 2010, 2015, 9999]
labels = ['before_2000', '2000_2005', '2005_2010', '2010_2015', 'after_2015']

# Dividir los años en lustros y crear variables dummies
df_games_release_lustrum['release_lustrum'] = pd.cut(df_games_release_lustrum['release_year'], bins=bins, labels=labels)
df_games_release_lustrum = pd.get_dummies(df_games_release_lustrum, columns=['release_lustrum'])
df_games_release_lustrum = df_games_release_lustrum.multiply(1)

# Eliminar la columna original de 'release_year'
df_games_release_lustrum.drop(columns=['release_year'], inplace=True)

# Vuelvo a realizar el SIST de RECO con estas modificaciones:
# OJO: Volver a cargar dfs

# Pondero categorías
df_games_release_lustrum[df_games_release_lustrum == 1] = 2 # Ponderación fuerte ya que es una sola columna con 1
#df_games_specs_v4[df_games_specs_v4 == 1] = 0.25
#df_games_tags_v4[df_games_tags == 1] = 1
#df_games_genres_v4[df_games_genres == 1] = 0.125

# Realiza un join de los DataFrames df1 y df2
merged_df_1 = pd.merge(df_games_names, df_games_genres_v4, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_2 = pd.merge(merged_df_1, df_games_specs_v4, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_3 = pd.merge(merged_df_2, df_games_release_lustrum, on='item_id', how='inner')

# Realiza un join de merged_df con df3
merged_df_final = pd.merge(merged_df_3, df_games_tags_v4, on='item_id', how='inner')

In [ ]:
# Índices de juegos conocidos para luego poder evaluar el desempeño del modelo

# Juego Counter Strike 32103
# Juego de futbol 32108
# Juego de baseball 6001
# Juego de Formula 1 7013
# Juego Worms 7027

In [47]:
games_dummies = merged_df_final.drop(columns=['item_id', 'app_name'])

n_neighbors=6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(games_dummies)

index = 7027
game_eval = np.array(games_dummies.iloc[index]).reshape(1,-1)
dif, ind = nneighbors.kneighbors(game_eval)

print("Juego seleccionado")
print("="*80)
print(df_games_names.loc[ind[0][0], :])
print("Juegos recomendados")
print("="*80)
df_games_names.loc[ind[0][1:], :]

Juego seleccionado
item_id        327030.0
app_name    Worms W.M.D
Name: 7027, dtype: object
Juegos recomendados


c:\Users\urtea\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,item_id,app_name
23084,435460.0,NotCoD™
15373,637745.0,Rocksmith® 2014 Edition – Remastered – Marilyn...
23506,463090.0,Hatoful Boyfriend: Holiday Star Collector's Ed...
23083,438680.0,One Troll Army
12070,617810.0,Total War: WARHAMMER II - Mortal Empires


In [285]:
# 8. Exporto en formato parquet el df final para implementar el Sistema de Recomendación V4

merged_df_final.to_parquet('df_sist_reco_v4.parquet')
